### **Práctica: Generación de Rostros con VAE usando CelebA**  
**Objetivo**: Entrenar un Variational Autoencoder (VAE) convolucional para generar rostros nuevos a partir del dataset CelebA.  

---

### **1. Configuración Inicial**  
**Objetivo**: Preparar el entorno y el dataset.  

**Instrucciones**:  

1. **Descargar CelebA**:  
   - El dataset contiene ~200k imágenes de rostros.  
   - Utilizamos `kagglehub` para descargar el dataset directamente desde Kaggle.

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jessicali9530/celeba-dataset")

print("Path to dataset files:", path)

/mnt/c/Users/aleja/Desktop/Master/05 - IA Generativa/Desarrollo/CasoEstudio-GenerarRostros/.venvWSL/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 1.33G/1.33G [00:32<00:00, 44.3MB/s]

Extracting files...


Path to dataset files: /home/aleja/.cache/kagglehub/datasets/jessicali9530/celeba-dataset/versions/2
